In [1]:
import numpy as np
import pandas as pd
from dLoader import DataLoader

In [2]:
# Load qyld csv
pd.read_csv('qyld_full-holdings_20220228.csv')

,% of Net Assets,Ticker,Name,SEDOL,Market Price ($),Shares Held,Market Value ($)
0,12.79,AAPL,APPLE INC,2046251,165.12,"5,059,138.00","835,364,866.56"
1,10.59,MSFT,MICROSOFT CORP,2588173,298.79,"2,315,126.00","691,736,497.54"
2,7.37,AMZN,AMAZON.COM INC,2000019,"3,071.26","156,642.00","481,088,308.92"
3,4.14,TSLA,TESLA INC,B616C79,870.43,"310,536.00","270,299,850.48"
4,4.05,NVDA,NVIDIA CORP,2379504,243.85,"1,085,828.00","264,779,157.80"
...,...,...,...,...,...,...,...
98,0.18,SWKS,SKYWORKS SOLUTIONS INC,2961053,138.17,"85,943.00","11,874,744.31"
99,0.18,DOCU,DOCUSIGN INC,BFYT7B7,118.43,"98,271.00","11,638,234.53"
100,0.17,PDD,PINDUODUO INC-ADR,BYVW0F7,51.86,"207,277.00","10,749,385.22"
101,0.14,SPLK,SPLUNK INC,B424494,118.10,"78,121.00","9,226,090.10"


In [3]:
qlyd = pd.read_csv('qyld_full-holdings_20220228.csv').dropna()

In [4]:
# Download the stock data
import os

folder = 'QLYD'
path = os.path.join(os.getcwd(), folder)

for ticker in qlyd.loc[:, 'Ticker']:
    try: 
        if not os.path.exists(os.path.join(path, ticker + '.csv')):
            DataLoader(ticker, dname='QLYD')
    except Exception as e:
        print(e)

# Analysis

### Gain Analysis

In [5]:
# Calculate Periodic Gains
period = 5
weeks = 4
dic = {}
for ticker in qlyd.loc[:, 'Ticker']:
    # Load price data form database
    data = DataLoader(ticker, dname='QLYD').get_data('2018-01-01', '2020-12-31')
    if len(data) < (250 * 3):
        continue
    # Turn in to periodic data
    batches = len(data) // period
    max_leng = batches * period
    close = np.array(data['Close'])[-max_leng:].reshape(-1, period)[:, -1]
    hClose = np.array(data['Close'])[-max_leng:].reshape(-1, period)[:, :-1].max(1)
    reshaped = pd.DataFrame(np.stack([close, hClose], axis=1), 
                            columns=['Close', 'High'])
    gains = (reshaped['High'].shift(-1) / reshaped['Close'] - 1).dropna()
    # Calculate total Gapital Gains
    total_cap_gain = np.prod(gains + 1)
    # Periodic Capital Gain
    w_batches = len(gains) // weeks
    w_max_leng = w_batches * weeks
    periodic = np.array(gains + 1)[-w_max_leng:].reshape(-1, weeks)
    periodic_cap_gain = np.prod(periodic, axis=1).mean()
    # Add to dictionary
    dic[ticker] = {'Total Capital Gain': total_cap_gain,
                   'Periodic Capital Gain': periodic_cap_gain,
                   'Mean Gain': gains.mean()}

In [6]:
df = pd.DataFrame.from_dict(dic, orient='index')

In [7]:
mask = df['Periodic Capital Gain'] > 1.1
picks = df[mask].sort_values(by='Periodic Capital Gain', ascending=False)

### Percentage Table

In [8]:
dic = {}
for ticker in picks.index:
    data = DataLoader(ticker).get_data('2018-01-01', '2020-12-31')
    # Turn in to periodic data
    batches = len(data) // period
    max_leng = batches * period
    opens = np.array(data['Open'])[-max_leng:].reshape(-1, period)[:, 0]
    close = np.array(data['Close'])[-max_leng:].reshape(-1, period)[:, -1]
    high = np.array(data['High'])[-max_leng:].reshape(-1, period).max(1)
    reshaped = pd.DataFrame(np.stack([opens, high, close], axis=1), 
                            columns=['Open', 'High', 'Close'])
    reshaped['Gains'] = reshaped['Close'].shift(-1) / reshaped['Close'] - 1
    reshaped['hGains'] = reshaped['High'].shift(-1) / reshaped['Close'] - 1
    reshaped['Gap'] = reshaped['Close'] / reshaped['Open'] - 1
    reshaped.dropna(inplace=True)
    mGap = np.round(reshaped['Gap'].mean(), 2)
    mGain = np.round(reshaped['Gains'].mean(), 2)
    # distrubution
    steps = np.arange(0.00, 0.101, step=0.005)
    # Gap Table
    gap_table = {}
    rang = np.unique(np.stack([mGap + steps,
                               mGap - steps]))
    for r in rang:
        mask = reshaped['Gap'] <= r
        if mask.sum() <= 0:
            gap_table[r] = {'Above': np.nan, 'Below': np.nan}
            continue
        gains = reshaped.loc[mask, 'Gains']
        gap_table[r] = {'Above': (gains > 0.).mean(),
                        'Below': (gains <= 0.).mean()}
    # Gain Table
    gain_table = {}
    rang = np.unique(np.stack([mGain + steps,
                               mGain - steps]))
    for r in rang:
        above = reshaped['hGains'] > r
        below = reshaped['hGains'] <= r
        gain_table[r] = {'Above': above.mean(),
                         'Below': below.mean()}
    # To DataFrame
    dic[ticker] = {'Mean Gain': mGain,
                   'Gap Table': pd.DataFrame.from_dict(gap_table, orient='index'),
                   'Gain Table': pd.DataFrame.from_dict(gain_table, orient='index')}

# Test

In [9]:
data = DataLoader('TSLA').get_data('2021-01-01', '2021-12-31')

In [26]:
# Reshape
batches = len(data) // period
max_leng = batches * period
opens = np.array(data['Open'])[-max_leng:].reshape(-1, period)[:, 0]
high = np.array(data['High'])[-max_leng:].reshape(-1, period).max(1)
close = np.array(data['Close'])[-max_leng:].reshape(-1, period)[:, -1]
reshaped = pd.DataFrame(np.stack([opens, high, close], axis=1), 
                        columns=['Open', 'High', 'Close'])
reshaped['Gap'] = reshaped['Close'] / reshaped['Open'] - 1
reshaped['Gain'] = reshaped['High'].shift(-1) / reshaped['Close'] - 1
reshaped['Base'] = reshaped['Close'].shift(-1) / reshaped['Close'] - 1
reshaped.dropna(inplace=True)
reshaped

,Open,High,Close,Gap,Gain,Base
0,758.489990,884.489990,849.440002,0.119909,0.015963,0.001189
1,852.760010,863.000000,850.450012,-0.002709,0.058734,0.016121
2,855.000000,900.400024,864.159973,0.010713,0.018909,-0.010959
3,820.000000,880.500000,854.690002,0.042305,0.027004,-0.058349
4,855.000000,877.770020,804.820007,-0.058690,0.031137,-0.021669
5,812.440002,829.880005,787.380005,-0.030845,0.011951,-0.133557
6,795.000000,796.789978,682.219971,-0.141862,0.057005,-0.089091
7,700.000000,721.109985,621.440002,-0.112229,0.155140,0.125772
8,626.059998,717.849976,699.599976,0.117465,0.019411,-0.066381
9,670.000000,713.179993,653.159973,-0.025134,0.071131,-0.019551


In [54]:
# Check if is a good buy
gap_table = dic['TSLA']['Gap Table']
gain_table = dic['TSLA']['Gain Table']
good_buy = []
for i, row in reshaped.iterrows():
    m = gap_table.index <= row['Gap']
    if m.sum() > 0:
        if gap_table[m].iloc[-1]['Above'] > .55:
            good_buy.append(True)
        else:
            good_buy.append(False)
    else:
        good_buy.append(False)

mask = np.array(good_buy)

In [55]:
score = reshaped.loc[mask, ['Gain', 'Base']].copy()
checks = score['Gain'] <= .01
gains = score['Gain'].copy()
gains[checks] = score.loc[checks, 'Base']
np.prod(gains + 1)

6.427867156874856